In [6]:
import sys
!conda install --yes --prefix {sys.prefix} pyserial
import serial
import time
import json

class tinyg(object):
  def __init__(self):
    self._response = ""
    self._rest = {}   # in this dictionary we store the RESTfull state of the tinyg.

  def open(self, p):
    try:
      self.ser = serial.Serial(port=p, baudrate=115200, timeout=0, xonxoff=False)
    except:
	    print ("Error opening serial port ", p)
	    sys.exit()
    self.handle_response()
    self.ser.write(str.encode('{"fv":""\n'))
    time.sleep(0.1)
    self.handle_response()

  def close(self):
      sys.exit()

  def move_x(self, x, speed = 200):
    gcode = "G01 X "+str(x)+ "F"+str(speed)
    self.send_gcode(gcode)

  def move_y(self, y, speed = 200):
    gcode = "G01 Y "+str(y)+ "F"+str(speed)
    self.send_gcode(gcode)

  def move_z(self, z, speed = 200):
    gcode = "G01 Z "+str(z)+ "F"+str(speed)
    self.send_gcode(gcode)

  def check_status(self) :
    status = None
    if "sr" in self._rest:
      sr = self._rest["sr"]
      if "stat" in sr:
        status = sr["stat"]
    return status

  def send_gcode(self,gcode):
    print ("sending gcode", gcode)
    self.ser.write( str.encode('{"gc":"' + gcode + '"}\n') )
    print ("waiting for status ok...")
    while 1:
     time.sleep(0.1)
     self.handle_response()  # ignore the return value
     if self.check_status() == 3:
        break
    print ("status ok")
  
  def handle_response(self):
    j = {}
    while self.ser.inWaiting() :
      c = self.ser.read()
      c = c.decode("utf-8")
      if( c == '\n' ) :
        print ("received a line", self._response)
        try:
          j = json.loads(self._response)
          # now merge the current response with the REST dictionary
          self._rest = dict(self._rest, **j)
        except:
          print ("error in json. ignoring line:", self._response)
        self._response = ""
      else:
         self._response += c; # append the character  
    return j

  def move_to_filter(self,num):
    x = 360*(num/44)
    return self.move_x(x,800)

if __name__ == "__main__":
  print ("connecting to the first available serial port")
  tg = tinyg()
  # tg.open( serial_ports()[0] )
  tg.open( "/dev/ttyUSB0" )
  #tg.close
  tg.move_to_filter(0)

Solving environment: done

# All requested packages already installed.

connecting to the first available serial port
received a line {"r":{"fv":0.970},"f":[1,0,9,8424]}
sending gcode G01 X 0.0F800
waiting for status ok...
received a line {"r":{},"f":[1,0,23,103]}
received a line {"sr":{"vel":0.01,"stat":5}}
received a line {"sr":{"posx":71.920,"vel":609.41}}
received a line {"sr":{"posx":65.644,"vel":800.00}}
received a line {"sr":{"posx":59.112}}
received a line {"sr":{"posx":52.581}}
received a line {"sr":{"posx":46.049}}
received a line {"sr":{"posx":39.517}}
received a line {"sr":{"posx":32.985}}
received a line {"sr":{"posx":26.520}}
received a line {"sr":{"posx":19.989}}
received a line {"sr":{"posx":13.457}}
received a line {"sr":{"posx":6.925}}
received a line {"sr":{"posx":1.005,"vel":463.83}}
received a line {"sr":{"posx":0.000,"vel":0.00,"stat":3}}
status ok
